In [ ]:
#figure out what files were skipped for/are missing from results

import os, shutil
import time
from os import listdir
from os.path import isfile, join


direct2= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits/'
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits2/'
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results/'
files = [f for f in listdir(direct2) if isfile(join(direct2,f)) ]
results =  [f for f in listdir(direct3) if isfile(join(direct3,f)) ]
files.sort()
pot_res=[]
for i in range(len(files)):
    temp='results'+str(i)+'.csv'
    pot_res.append(temp)
missing = [name for name in pot_res if name not in results]
# print(missing)
filestorun=[]
for i in missing:
    temp= i[:-4]
    temp=int(temp[7:])
    filestorun.append(files[temp])

for i in filestorun:
    # print(i)
    full_file_name = os.path.join(direct2,i)
    # print(full_file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name,new_splits)

# print(files)
# print(results)

In [ ]:

import os, shutil
import time
from os import listdir
from os.path import isfile, join
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits2/'
filestorun = [f for f in listdir(new_splits) if isfile(join(new_splits,f)) ]
print(filestorun)

In [ ]:

import os, shutil
import time
from os import listdir
from os.path import isfile, join
#60 sequences per line
# max of 1000 sequences per entry
# max of 16 lines
#take each file, split proteins down to sequences of less than 1000 and make each of them a new file
new_splits= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits4/'
split_splits='C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human/splits5/'
filestorun = [f for f in listdir(new_splits) if isfile(join(new_splits,f)) ]
print(filestorun)
split_lines=16
linemax=150
string1 = '>'
i=1100
#step through current splits of the original fasta file and separate them into smaller splits if need be
for file in filestorun:
    lineindex = 0
    protind=0


    tempfile = open(new_splits+file, 'r')
    tempnew = open(split_splits+'human'+str(i)+'.fasta', 'w')


    for line in tempfile:
        lineindex +=1
        protind +=1
        if string1 in line:
            protind=0
            header=line
        if protind>split_lines:
            i += 1
            lineindex = 0
            tempnew.close()
            tempnew = open(split_splits+'human'+str(i)+'.fasta', 'w')
            tempnew.write(header)
            protind=0
        tempnew.write(line)
    tempfile.close()
    tempnew.close()
    i+=1
    print(i)

In [ ]:
#cleanup output csvs
#any protein with a comma in its name is messed up
#pattern if the line does not end with a comma then remove the first comma after opening as a txt
import os
import time
from os import listdir
from os.path import isfile, join
# file1=open('results0_test.csv','r')
# file2=open('results0_clean.csv', 'w')
direct1= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results5/'
direct2='C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results5/Cleaner_results/'
files = [f for f in listdir(direct1) if isfile(join(direct1,f)) ]
for i in range(len(files)):
    print(i)
    file1=open(direct1+files[i],'r')
    file2=open(direct2+'results'+str(i)+'.csv.','w')

    try:
        for line in file1:
#     print(line.count(','))
            while line.count(',')>20:
                line=line.replace(',','',1)
            # print(line.count(','))
            file2.write(line)
    except Exception:
        print('tried file'+ str(i)+',but it didnt work or may be empty')
        pass
    file1.close()
    file2.close()

In [ ]:
#Code for processing NSP3 output for Mouse Proteome

from NSP_functions import *
from os import listdir
from os.path import isfile, join
import csv
import hashlib

direct2= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/'
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/Proc_Results/'
files = [f for f in listdir(direct2) if isfile(join(direct2,f)) ]
descript= 'proc_results_3B'
# print(files)
processed_data=pd.DataFrame()
for i in range(len(files)):
# for i in range(1):
    filename=direct2+files[i]
    unique_lines= set()
    # print('check2')
    with open(filename, newline='') as csvfile:
        reader= csv.reader(csvfile,delimiter=' ', quotechar='|')
        for row in reader:
            x=row[0]
            y=x[4:10]
            hashval=hashlib.md5(y.rstrip().encode('utf-8')).hexdigest()
            if hashval not in unique_lines:
                unique_lines.add(y)
    uniqueIds=list(unique_lines)
    print('complete')
    # print(uniqueIds[1:])

    try:
        raw_netsurfp_data = pd.read_csv(filename)
        raw_netsurfp_data.columns = raw_netsurfp_data.columns.str.strip()
        tmp_processed_data = netsurfp_2_data_processing(uniqueIds[1:], raw_netsurfp_data)
    except Exception:
        print('skipped at'+str(i))
        pass
    processed_data=pd.concat([processed_data,tmp_processed_data],axis=0)
processed_data.to_excel(direct3+descript+str(i)+'.xlsx')
    # processed_data.to_excel(direct3+'aaaaa'+str(i)+'.xlsx')

In [ ]:
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/Proc_Results/'
processed_data.to_excel(direct3+descript+str(i)+'.xlsx')

In [ ]:
#Combine all processed files into one complete proteome file for mouse Proteome
import glob
import pandas as pd

direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results4/Cleaner_results/D/'
file_list = glob.glob(direct3 + "/*.csv")

excl_list =[]

for file in file_list:

    try:
        excl_list.append(pd.read_csv(file))
    except Exception:
        print(file)
excl_merged=pd.DataFrame()

for excl_file in excl_list:
    excl_merged= pd.concat([excl_merged,excl_file] , ignore_index=True)

excl_merged.to_csv(direct3+'combined_clean_results4D.csv', index= False)

In [10]:
#cleanup output csvs
#any protein with a comma in its name is messed up
#pattern if the line does not end with a comma then remove the first comma after opening as a txt
import os
import time
from os import listdir
from os.path import isfile, join
# file1=open('results0_test.csv','r')
# file2=open('results0_clean.csv', 'w')
direct1= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results5/'
direct2='C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results5/Cleaner_results/'
files = [f for f in listdir(direct1) if isfile(join(direct1,f)) ]
for i in range(len(files)):
    print(i)
    file1=open(direct1+files[i],'r')
    file2=open(direct2+'results'+str(i)+'.csv.','w')

    try:
        for line in file1:
#     print(line.count(','))
            while line.count(',')>20:
                line=line.replace(',','',1)
            # print(line.count(','))
            file2.write(line)
    except Exception:
        print('tried file'+ str(i)+',but it didnt work or may be empty')
        pass
    file1.close()
    file2.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93


In [12]:
#Code for processing NSP3 output for Mouse Proteome

from NSP_functions import *
from os import listdir
from os.path import isfile, join
import csv
import hashlib

direct2= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/'
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/Proc_Results/'
files = [f for f in listdir(direct2) if isfile(join(direct2,f)) ]
descript= 'proc_results_3B'
# print(files)
processed_data=pd.DataFrame()
for i in range(len(files)):
# for i in range(1):
    filename=direct2+files[i]
    unique_lines= set()
    # print('check2')
    with open(filename, newline='') as csvfile:
        reader= csv.reader(csvfile,delimiter=' ', quotechar='|')
        for row in reader:
            x=row[0]
            y=x[4:10]
            hashval=hashlib.md5(y.rstrip().encode('utf-8')).hexdigest()
            if hashval not in unique_lines:
                unique_lines.add(y)
    uniqueIds=list(unique_lines)
    print('complete')
    # print(uniqueIds[1:])

    try:
        raw_netsurfp_data = pd.read_csv(filename)
        raw_netsurfp_data.columns = raw_netsurfp_data.columns.str.strip()
        tmp_processed_data = netsurfp_2_data_processing(uniqueIds[1:], raw_netsurfp_data)
    except Exception:
        print('skipped at'+str(i))
        pass
    processed_data=pd.concat([processed_data,tmp_processed_data],axis=0)
processed_data.to_excel(direct3+descript+str(i)+'.xlsx')
    # processed_data.to_excel(direct3+'aaaaa'+str(i)+'.xlsx')

complete


C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is t

complete


C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is t

complete


C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.loc[:, 'class assignment'] = np.where(filtered.loc[:, 'rsa'] > 0.25, 'E', 'B')
C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\NSP_functions.py:221: SettingWithCopyWarning: 
A value is t

In [10]:
direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_proc_results/Proc_Results/'
processed_data.to_excel(direct3+descript+str(i)+'.xlsx')

In [11]:
#Combine all processed files into one complete proteome file for mouse Proteome
import glob
import pandas as pd

direct3= 'C:/Users/pouls/OneDrive - Duke University/NSP3_Working_stuff/Human_results4/Cleaner_results/D/'
file_list = glob.glob(direct3 + "/*.csv")

excl_list =[]

for file in file_list:

    try:
        excl_list.append(pd.read_csv(file))
    except Exception:
        print(file)
excl_merged=pd.DataFrame()

for excl_file in excl_list:
    excl_merged= pd.concat([excl_merged,excl_file] , ignore_index=True)

excl_merged.to_csv(direct3+'combined_clean_results4D.csv', index= False)